In [2]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.datasets import mnist

import matplotlib.pyplot as plt


def generator_model():
    dropout = 0.4
    dim = 128
    depth = 512
    # In: 100
    # Out: 128 x 512
    model = Sequential()
    model.add(Dense(dim*depth, input_dim=100))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Reshape((dim, depth)))
    model.add(Dropout(dropout))
    # In: 128 x 512
    # Out: 256 x 256
    model.add(UpSampling1D())
    model.add(Conv1DTranspose(int(depth/2), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 256 x 256
    # Out: 512 x 128
    model.add(UpSampling1D())
    model.add(Conv1DTranspose(int(depth/4), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 512 x 128
    # Out: 512 x 1
    model.add(Conv1DTranspose(int(depth/8), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Conv1DTranspose(1, 5, padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    return model


def discriminator_model():
    depth = 64
    dropout = 0.4
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth = 64
    
#     model.add(Conv2D(depth, 5, strides = 2, padding ='same', input_shape=(28, 28, 1)))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(dropout))
    
#     model.add(Conv2D(depth*2, 5, strides = 2, padding ='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(dropout))
    
#     model.add(Conv2D(depth*4, 5, strides = 2, padding ='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(dropout))
    
#     model.add(Conv2D(depth*8, 5, strides = 1, padding ='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(dropout))

    model.add(Dense(2 ** l1, activation = 'relu', input_shape = (512,)))
    model.add(Dropout(0.5))
    model.add(Dense(2 ** l1, activation = 'relu'))
    model.add(Dropout(0.5))
    
    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    return model

def adversial_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

def train(epochs = 1000, batch_size = 256, save_interval = 50):
    # Initialize data and models
#     (X_train, y_train), (X_test, y_test) = mnist.load_data()
#     X_train = input_data.read_data_sets("mnist",one_hot=True).train.images
#     X_train = X_train.reshape(-1, 28, 28, 1).astype(np.float32)
    discriminator = discriminator_model()
    generator = generator_model()
    adversial = adversial_model(generator, discriminator) # discriminator is frozen
    d_optim = RMSprop(lr=0.0002, decay=6e-8)
    g_optim = RMSprop(lr=0.0001, decay=3e-8)
    adversial.compile(loss='binary_crossentropy', optimizer=g_optim, metrics=['accuracy'])
    discriminator.trainable = True # unfreeze discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim, metrics=['accuracy'])
    
    # Stuff for saving images
    filename = 'mnist.png'
    noise_input = None
    if save_interval > 0:
        noise_input = np.random.uniform(-1, 1, size=[16,100])
    
    # The training loop
    for epoch in range(epochs):
        real_images = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        fake_images = generator.predict(noise)
        
        # Train fake and real images together (could also train separately, unsure which is better)
        combined_images = np.concatenate((real_images, fake_images))
        labels = np.ones([2*batch_size, 1])
        labels[batch_size:, :] = 0
#         labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
#         labels += 0.05 * np.random.random(labels.shape)

        # Train discriminator
        d_loss = discriminator.train_on_batch(combined_images, labels)
                                        
            
        misleading_targets = np.ones([batch_size, 1])
        random_latent_vectors = np.random.uniform(-1, 1, size=[batch_size, 100])
        discriminator.trainable = False # freeze the discriminator
        
        #train adversial network with d freezed 
        a_loss = adversial.train_on_batch(random_latent_vectors, misleading_targets)
        
        discriminator.trainable = True # unfreeze the discrminator
        
                                        
        log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
                       
        # Saving sample generated images
        if save_interval>0:
            step = epoch+1
            if step%save_interval==0:
                filename = "mnist_%d.png" % step
                images = generator.predict(noise_input)
                plt.figure(figsize=(10,10))
                for i in range(images.shape[0]):
                    plt.subplot(4, 4, i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image, [28, 28])
                    plt.imshow(image, cmap='gray')
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig(filename)
                plt.close('all')
train()

Extracting mnist\train-images-idx3-ubyte.gz
Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None

KeyboardInterrupt: 